<a href="https://colab.research.google.com/github/natdebandi/hate_speech_ar/blob/main/3_GPT_finetuning_multiclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP final - reconocimiento de discursos discriminatorios en Twitter
## (3 )Finetuning de GPT 4 mini para clasificación binaria

**Natalia Dedandi**



La documentación de la API se encuentra aqui:

https://platform.openai.com/docs/api-reference/introduction


Guia para finetuning:

https://medium.com/@garethcull/fine-tuning-ai-models-a-practical-guide-for-beginners-dc313b2e0f76

In [8]:
pip install openai

In [9]:
!pip install datasets seaborn


In [10]:
import json
from openai import OpenAI
import os
from google.colab import userdata
#seteo la KEY
os.environ['OPENAI_API_KEY'] = userdata.get('openIA_key')




In [11]:
#creo el cliente OPENAI con mi usuario y proyecto
client = OpenAI(
  organization='org-1uHjwiaB3OlPzoxfVzhqOSzs',
  project='proj_2fII7izwVGgYaERNNKhhMx4l',
)

Levanto los datos de entrenamiento

In [12]:
from datasets import load_dataset
import pandas as pd

ds1 = load_dataset("piuba-bigdata/contextualized_hate_speech")


Tengo que transformar el DS de train en un conjunto de mensajes para pasarle a GPT para finetuning.

In [22]:
# prompt: Create a dataframe from ds1 with id text and hateful cols

df_train = pd.DataFrame(ds1['train'])[['id', 'text', 'HATEFUL']]


In [23]:
# prompt: create a datfram from ds1 for train
df_train

,id,text,HATEFUL
0,343726,@usuario Uno menos,1
1,384799,@usuario #QueSeVayanTodos @usuario @usuario @u...,0
2,399435,"@usuario Te falta negociar con Cuba, mamerto.",0
3,403171,@usuario Se prepara para las milicias armadas,0
4,394186,@usuario Seguimos alimentando vagos,1
...,...,...,...
36415,402634,@usuario Que se vaya a c...\n.,0
36416,340909,@usuario Me parece excelente,1
36417,404621,@usuario fuaahh todo es acoso . pobre de la ge...,0
36418,342595,@usuario Si vas a jugar al póker todas las noc...,0


In [24]:
df_train['hateful_label'] = df_train['HATEFUL'].apply(lambda x: "hateful" if x==1  else "not hateful")
df_train

,id,text,HATEFUL,hateful_label
0,343726,@usuario Uno menos,1,hateful
1,384799,@usuario #QueSeVayanTodos @usuario @usuario @u...,0,not hateful
2,399435,"@usuario Te falta negociar con Cuba, mamerto.",0,not hateful
3,403171,@usuario Se prepara para las milicias armadas,0,not hateful
4,394186,@usuario Seguimos alimentando vagos,1,hateful
...,...,...,...,...
36415,402634,@usuario Que se vaya a c...\n.,0,not hateful
36416,340909,@usuario Me parece excelente,1,hateful
36417,404621,@usuario fuaahh todo es acoso . pobre de la ge...,0,not hateful
36418,342595,@usuario Si vas a jugar al póker todas las noc...,0,not hateful


Creo una lista BATCH con el formato de mensajes que debo enviar a GPT para finetunning

In [38]:


  # Convert the dataframe into a list of dictionaries with custom_id and messages
  batch = [
      {'messages': [
                  {'role': 'system', 'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
                  {'role': 'user', 'content': row['text']},
                  {'role': 'assistant', 'content': row['hateful_label']}
                  ]
       }
      for _, row in df_train.iterrows()
  ]


In [40]:
batch[0]

{'messages': [{'role': 'system',
   'content': 'You are trained to analyze and detect the sentiment of the given text.You must classify a text with one or more of the following labels: WOMEN when there is discrimination against women, LGBTI when there is discrimination against gays, lesbians or transsexuals, RACISM when there is discrimination against people according to their skin color, CLASS  when there is discrimination for being poor or living in a marginal or poor place, POLITICS is hatred of politicians, DISABLED when there is discrimination against people with disabilities, APPEARENCE when there is discrimination against people because of their physical appearance, specifically fatphobia, CRIMINAL when there is discrimination against people because of a criminal record or links to criminality and  CALLS is hateful text that incites violence. If the text is not hateful indicate NOT HATEFUL'},
  {'role': 'user', 'content': '@usuario Uno menos'},
  {'role': 'assistant', 'content':

In [41]:
# Save the batch to a .jsonl file
with open('batch_gpt_bin_train.jsonl', 'w') as f:
    for entry in batch:
        json.dump(entry, f)
        f.write('\n')

print("Batch file created successfully in .jsonl format!")

Batch file created successfully in .jsonl format!


Ya tengo creado el archivo y lo subo para finetunning

In [42]:
from openai import OpenAI
client = OpenAI()

client.files.create(
  file=open("batch_gpt_bin_train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-I6srp0hpoIQllyxQRQ4U24iB', bytes=39613877, created_at=1722719183, filename='batch_gpt_bin_train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [43]:
client.fine_tuning.jobs.create(
  training_file="file-I6srp0hpoIQllyxQRQ4U24iB",
  model="gpt-3.5-turbo-0125",
  hyperparameters={
    "n_epochs":5
  },
  suffix="hate_speech_v2"
)

FineTuningJob(id='ftjob-3hLvVaaM5T51D19lXitFUC5Q', created_at=1722719196, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=1064557800, status='validating_files', trained_tokens=None, training_file='file-I6srp0hpoIQllyxQRQ4U24iB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')

In [46]:
fine_tuning_job = client.fine_tuning.jobs.retrieve(
  fine_tuning_job_id="ftjob-3hLvVaaM5T51D19lXitFUC5Q"
)

print(fine_tuning_job)


FineTuningJob(id='ftjob-3hLvVaaM5T51D19lXitFUC5Q', created_at=1722719196, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=5, batch_size=121, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-1uHjwiaB3OlPzoxfVzhqOSzs', result_files=[], seed=1064557800, status='validating_files', trained_tokens=None, training_file='file-I6srp0hpoIQllyxQRQ4U24iB', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix='hate_speech_v2')
